In [1]:
from websocket import create_connection
import json
import pandas as pd

socket = "wss://data.tradingview.com/socket.io/websocket"
ws = create_connection(socket)

In [2]:
def create_msg(fun, arg):
    ms = json.dumps({"m": fun, "p": arg})
    msg = "~m~" + str(len(ms)) + "~m~" + ms
    print(msg)

In [3]:
create_msg("chart_create_session", ["cs_Z8lZUkSJ1hd0", ""])
create_msg("resolve_symbol", ["cs_Z8lZUkSJ1hd0","sds_sym_1","={\"adjustment\":\"splits\",\"currency-id\":\"USD\",\"symbol\":\"NYMEX:CL1!\",\"unit-id\":\"BLL\"}"])
create_msg("create_series", ["cs_Z8lZUkSJ1hd0","sds_1","s1","sds_sym_1","D",300,""])

~m~59~m~{"m": "chart_create_session", "p": ["cs_Z8lZUkSJ1hd0", ""]}
~m~164~m~{"m": "resolve_symbol", "p": ["cs_Z8lZUkSJ1hd0", "sds_sym_1", "={\"adjustment\":\"splits\",\"currency-id\":\"USD\",\"symbol\":\"NYMEX:CL1!\",\"unit-id\":\"BLL\"}"]}
~m~90~m~{"m": "create_series", "p": ["cs_Z8lZUkSJ1hd0", "sds_1", "s1", "sds_sym_1", "D", 300, ""]}


In [4]:
# while True:
#     res = ws.recv()
#     print(res)

In [5]:
import base64
import json

from bytewax.dataflow import Dataflow
from bytewax.inputs import ManualInputConfig, distribute


from websocket import create_connection

## input
ticker_list = ['AMZN', 'MSFT']

def yf_input(worker_tickers, state):
        ws = create_connection("wss://streamer.finance.yahoo.com/")
        ws.send(json.dumps({"subscribe": worker_tickers}))
        while True:
            yield state, ws.recv()


def input_builder(worker_index, worker_count, resume_state):
    state = resume_state or None
    worker_tickers = list(distribute(ticker_list, worker_index, worker_count))
    print({"subscribing to": worker_tickers})
    return yf_input(worker_tickers, state)


flow = Dataflow()
flow.input("input", ManualInputConfig(input_builder))

ImportError: cannot import name 'ManualInputConfig' from 'bytewax.inputs' (/Users/helenafu/miniconda3/envs/trading_explore/lib/python3.10/site-packages/bytewax/inputs.py)

In [ ]:
from ticker_pb2 import Ticker


def deserialize(message):
    '''Use the imported Ticker class to deserialize 
    the protobuf message

    returns: ticker id and ticker object
    '''
    ticker_ = Ticker()
    message_bytes = base64.b64decode(message)
    ticker_.ParseFromString(message_bytes)
    return (ticker_.id, ticker_)
    
flow.map(deserialize)